# NGC 3568 DeepCR Remover

I have had some trouble removing CRs with AstroScrappy so this is an attempt to remove CRs with DeepCR instead. This should be run instead of the *Remove FLC CRs with Astroscrappy* in [NGC3568-ImageReducer.ipynb](NGC3568-ImageReducer.ipynb#remove_flc_crs_with_astroscrappy).

## Imports

In [ ]:
# Python Imports
# import os
from os import path
from glob import glob, iglob

# Astropy Colab Imports
from astropy.io import fits

# Other Astronomy Imports
from deepCR import deepCR

## Notebook Setup

In [ ]:
# Conda Dir
CONDA_DIR = path.expanduser('~/Software/miniconda3')
MDL_DIR   = path.join(CONDA_DIR, 'envs/dcr/lib/python3.11/site-packages/learned_models')

In [ ]:
# Data Directory
DATA_DIR = 'mastDownload/HST'

# FLC Glob Patter
FLC_GLOB_PAT    = path.join(DATA_DIR, '**/*[!_cr]_flc.fits')

# Backup Name
BACKUP_NAME = path.join(DATA_DIR, 'FLCs.tar.gz')

## Store File Name Data

In [ ]:
# Get the File Names and Sort them by filter
fileNameDict = {}
for fn in iglob(FLC_GLOB_PAT):
    
    # Open the file to get the filter
    with fits.open(fn) as hduList:
        hdr = hduList[0].header  # Get the Header
        if 'FILTER' in hdr:      # If the FILTER keyword exists (WFC3)
            filt = hdr['FILTER']
        elif 'CLEAR' not in hdr['FILTER1']:  # If FILTER1 is not clear (ACS)
            filt = hdr['FILTER1']
        else:                                # Else FILTER2 must be the filter (ACS)
            filt = hdr['FILTER2']
    
    # Store the Name using the filter as the dict key
    # Start the Empty List if Key does not exist
    if filt not in fileNameDict:
        fileNameDict[filt] = []
    fileNameDict[filt].append(fn)

## Remove CRs

### Remove F814W CRs

In [ ]:
# Create the Model
mdl = deepCR(
    mask=path.join(MDL_DIR, 'mask/ACS-WFC-F814W.pth'),
    device='CPU'
)

In [ ]:
# Loop Through Files
for fn in fileNameDict['F814W']:
    
    # Out Name
    outName = fn.replace('flc.fits', 'deep_cr_flc.fits')
    
    # Open Image
    with fits.open(fn) as hduList:
        
        # Get Cleaned Arrays
        crmsk1, crarr1 = mdl.clean(
            hduList['SCI', 1].data, #threshold=0.4,
            binary=True,
            inpaint=True, n_jobs=2)
        crmsk2, crarr2 = mdl.clean(
            hduList['SCI', 2].data, #threshold=0.4,
            binary=True,
            inpaint=True, n_jobs=2)
        
        # Write the Output Data
        outList = hduList.copy()  # Copy the Original File
        outList[0].header.add_history('CRs removed with DeepCR')  # Update comment about processing
        outList['SCI', 1].data = crarr1  # Change the old SCI image for CR Removed img
        outList.insert(4, fits.ImageHDU(crmsk1.astype('uint8'), hduList['SCI', 1].header, 'MSK'))
        outList['SCI', 2].data = crarr2
        outList.insert(8, fits.ImageHDU(crmsk2.astype('uint8'), hduList['SCI', 2].header, 'MSK'))
        outList.writeto(outName, overwrite=True)

### Remove F475W CRs

In [ ]:
# Create the Model
mdl = deepCR(
    mask=path.join(MDL_DIR, 'mask/ACS-WFC-F435W.pth'),
    device='CPU'
)

In [ ]:
# Loop Through Files
for fn in fileNameDict['F475W']:
    
    # Out Name
    outName = fn.replace('flc.fits', 'deep_cr_flc.fits')
    
    # Open Image
    with fits.open(fn) as hduList:
        
        # Get Cleaned Arrays
        crmsk1, crarr1 = mdl.clean(
            hduList['SCI', 1].data,
            binary=True,
            inpaint=True, n_jobs=2
        )
        crmsk2, crarr2 = mdl.clean(
            hduList['SCI', 2].data,
            binary=True,
            inpaint=True, n_jobs=2
        )
        
        # Write the Output Data
        outList = hduList.copy()  # Copy the Original File
        outList[0].header.add_history('CRs removed with DeepCR')  # Update comment about processing
        outList['SCI', 1].data = crarr1  # Change the old SCI image for CR Removed img
        outList.insert(4, fits.ImageHDU(crmsk1.astype('uint8'), hduList['SCI', 1].header, 'MSK'))
        outList['SCI', 2].data = crarr2
        outList.insert(8, fits.ImageHDU(crmsk2.astype('uint8'), hduList['SCI', 2].header, 'MSK'))
        outList.writeto(outName, overwrite=True)

### Remove F275W CRs

In [ ]:
# Create the Model
mdl = deepCR(
    mask=path.join(MDL_DIR, 'mask/ACS-WFC-F435W.pth'),
    device='CPU'
)

In [ ]:
# Loop Through Files
for fn in fileNameDict['F275W']:
    
    # Out Name
    outName = fn.replace('flc.fits', 'deep_cr_flc.fits')
    
    # Open Image
    with fits.open(fn) as hduList:
        
        # Get Cleaned Arrays
        crmsk1, crarr1 = mdl.clean(hduList['SCI', 1].data, inpaint=True, n_jobs=2)
        crmsk2, crarr2 = mdl.clean(hduList['SCI', 2].data, inpaint=True, n_jobs=2)
        
        # Write the Output Data
        outList = hduList.copy()  # Copy the Original File
        outList[0].header.add_history('CRs removed with DeepCR')  # Update comment about processing
        outList['SCI', 1].data = crarr1  # Change the old SCI image for CR Removed img
        outList.insert(4, fits.ImageHDU(crmsk1.astype('uint8'), hduList['SCI', 1].header, 'MSK'))
        outList['SCI', 2].data = crarr2
        outList.insert(8, fits.ImageHDU(crmsk2.astype('uint8'), hduList['SCI', 2].header, 'MSK'))
        outList.writeto(outName, overwrite=True)